In [1]:
import os
import sys
import hotspot
import anndata
import numpy as np
import pandas as pd
import hotspot
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import mplscience
from scipy.io import mmread
from scipy.sparse import csr_matrix, csc_matrix
import scanpy as sc
import warnings; warnings.simplefilter('ignore')
import pickle5 as pickle
import mechanicalsoup
from time import time, sleep
import re
from datetime import datetime
import json
import itertools
from random import random

In [2]:
def method_in_2_lines(method):
    return method.replace(' (', '\n(')

In [3]:
ANALYSIS_FOLDER = "../SPIRAL/static/analysis/"
def filter_struct_lst_for_result_panel(data_n, new_Jaccard_thr_genes):
    # produces a list of structures, in which there is no pair of structures with Jaccard index larger than
    # Jaccard_thr_genes. The function relies on the fact that the structure numbers are
    # range(1, len(Jaccard_mat_genes) + 1), and that structure i corresponds to row i-1 in Jaccard_mat_genes.
    print('filter_struct_lst_for_result_panel!!!')

    data_path = os.path.join(ANALYSIS_FOLDER, 'data' + str(data_n))
    # get the Jaccard matrix of gene lists
    Jaccard_mat_genes = np.load(os.path.join(data_path, 'Jaccard_mat_genes.npy'))

    inds = list(range(1, len(Jaccard_mat_genes) + 1))
    thr_struct_lst = []

    while inds:
        i = inds.pop(0)
        thr_struct_lst.append(i)
        similar_to_i = []

        for j in inds:
            if Jaccard_mat_genes[i - 1][j - 1] >= new_Jaccard_thr_genes:
                similar_to_i.append(j)

        inds = [j for j in inds if j not in similar_to_i]

    return thr_struct_lst

In [4]:
# Build dataset_dict
dataset_dict = dict(zip([1, 3, 2, 5, 4, 6], 
                        ['lymphoblastoid cell line GM12891', 
                        'Zebrafish differentiation (7 time points)',
                        'sagittal-posterior section of a mouse brain',
                        'human prostate',
                        'human differentiation (days 0 and 7 − 21)',
                        'mouse B cells, treated with anti-IgM mAb (25 time points in the 6 hours post stimulation)'
                       ]))
dataset_type = dict(zip([1, 3, 2, 5, 4, 6], 
                        ['single cell', 'single cell', 
                        'spatial', 'spatial',
                        'bulk', 'bulk'
                       ]))

dataset_dict[20] = 'Slide-seqV2- diabetic kidney mouse'
dataset_type[20] = 'spatial'

dataset_dict[21] = 'Slide-seqV2- normal kidney mouse'
dataset_type[21] = 'spatial'

for data_n in range(61, 81):
    f = open("../SPIRAL/static/analysis/data" + str(data_n) + "/dataset_name.txt", 'r')
    name = f.readlines()[0]
    #print(name)
    f.close()
    
    dataset_dict[data_n] = 'mouse ' + name.replace('__', ' ')
    dataset_type[data_n] = 'spatial'


# Build methods_dict
methods_dict = {'single cell': ['SPIRAL (Jaccard index=0.05)',
                                'SPIRAL (Jaccard index=0.5)',
                                'SPIRAL (Jaccard index=0.75)',
                                'Seurat',
                                'Slingshot+tradeSeq (nPoints=20)',
                                'Slingshot+tradeSeq (nPoints=200)',
                                "Hotspot (n_neighbors=30)",
                                "Hotspot (n_neighbors=300)",
                                'nsNMF',
                                'SingleCellHaystack (k=3)',
                                'SingleCellHaystack (k=5)'],
               'spatial':  ['SPIRAL (Jaccard index=0.05)',
                            'SPIRAL (Jaccard index=0.5)',
                            'SPIRAL (Jaccard index=0.75)',
                            'Seurat',
                            "Hotspot (n_neighbors=30)",
                            "Hotspot (n_neighbors=300)",
                            'nsNMF',
                            'SpatialDE (C=3)',
                            'SpatialDE (C=5)',
                            'SingleCellHaystack (k=3)',
                            'SingleCellHaystack (k=5)'],
               'bulk': ['SPIRAL (Jaccard index=0.05)',
                        'SPIRAL (Jaccard index=0.5)',
                        'SPIRAL (Jaccard index=0.75)',
                        'Seurat',
                        'nsNMF']}


In [5]:
out_folder = "../SPIRAL_for_paper_/comparison_of_all_methods_real_datasets/module_sizes/"
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

In [13]:
n_participating_genes = dict()
data_n = 2
dataset = dataset_dict[data_n]
typee = dataset_type[data_n]
methods = methods_dict[typee]

print(dataset)
for method in methods:
    print(dataset, typee, method)
    
    data_path = "../SPIRAL/static/analysis/data" + str(data_n)
    species = open(os.path.join(data_path, 'species.txt'), "r").read()
    impute_method = open(os.path.join(data_path, 'imputation_method.txt'), "r").read()

    #get genelist for this dataset
    #genelist_file = os.path.join(data_path, impute_method + '_genetable.p')
    # Load genes_table
    #with open(genelist_file, 'rb') as fp:
    #    genes_table = pickle.load(fp)
    #background_list = str(list(genes_table.index)).replace("'", "").replace('[', '').replace(']', '').replace(', ','\n')

    flag = False
    if method == 'nsNMF':
        method_folder = 'nsNMF/data' + str(data_n) + '/'
        name_clus2 = 'module'
        for file in os.listdir(method_folder):
            if file.startswith("cluster_table"):
                #print(file)
                method_cluster_table_file = os.path.join(method_folder, file)
                flag = True
    elif method == 'Seurat':
        method_folder = 'Seurat_DE_genes/data' + str(data_n) + '/'
        name_clus2 = 'DE gene set'
        for file in os.listdir(method_folder):
            if file.startswith("cluster_table"):
                #print(file)
                method_cluster_table_file = os.path.join(method_folder, file)
                flag = True
    elif method in ['Slingshot+tradeSeq (nPoints=20)', 'Slingshot+tradeSeq (nPoints=200)']:
        method_folder = 'Slingshot/data' + str(data_n) + '/'
        name_clus2 = 'gene cluster'
        if method == 'Slingshot+tradeSeq (nPoints=20)':
            file = 'cluster_table_20.csv'
            method_cluster_table_file = os.path.join(method_folder, file)
            flag = True
        elif method == 'Slingshot+tradeSeq (nPoints=200)':
            file = 'cluster_table_200.csv'
            method_cluster_table_file = os.path.join(method_folder, file)
            flag = True
    elif 'Hotspot' in method:
        if data_n == 3:
            flag = False
        else:
            method_folder = 'Hotspot/data' + str(data_n) + '/'
            name_clus2 = 'gene module'
            if method == "Hotspot (n_neighbors=30)":
                file = 'cluster_table_danb_30.csv'     
            elif method == "Hotspot (n_neighbors=300)":
                file = 'cluster_table_danb_300.csv'
            method_cluster_table_file = os.path.join(method_folder, file)
            flag = True
    elif 'SpatialDE' in method:
        method_folder = 'SpatialDE/data' + str(data_n) + '/'
        name_clus2 = 'pattern'
        if method=='SpatialDE (C=3)':
            file = 'cluster_table_3.csv'
        elif method=='SpatialDE (C=5)':
            file = 'cluster_table_5.csv'
        method_cluster_table_file = os.path.join(method_folder, file)
        flag = True
    elif 'SingleCellHaystack' in method:
        method_folder = 'singlecellhaystack/data' + str(data_n) + '/'
        name_clus2 = 'cluster'
        if method=='SingleCellHaystack (k=3)':
            file = 'data' + str(data_n) + '.singleCellHaystack.k_3.csv'
        elif method=='SingleCellHaystack (k=5)':
            file = 'data' + str(data_n) + '.singleCellHaystack.k_5.csv'
        method_cluster_table_file = os.path.join(method_folder, file)
        flag = True


    if flag:
        method_cluster_table = pd.read_csv(method_cluster_table_file, index_col=0)

        if method in ['Seurat', 'Slingshot+tradeSeq (nPoints=20)', 'Slingshot+tradeSeq (nPoints=200)']:
            method_cluster_table.loc[:, 'genes'] = [
                str(genes.replace("['", "").replace("']", "").split(", ")) for genes in method_cluster_table.loc[:, 'genes']]
        elif 'SingleCellHaystack' in method:
            method_cluster_table = pd.read_csv(method_cluster_table_file)
            method_cluster_table.columns = ['#genes', 'genes']
            method_cluster_table.loc[:, 'genes'] = [
                str(a.split(' ')) for a in method_cluster_table.loc[:, 'genes']]
        print('\n\n', method, method_cluster_table_file)

    elif method in ['SPIRAL (Jaccard index=0.05)', 
                    'SPIRAL (Jaccard index=0.5)', 
                    'SPIRAL (Jaccard index=0.75)']:
        flag = True

        struct_table_file = os.path.join(data_path, impute_method + '_sigtable_filt_GO_vis.xlsx')

        struct_table = pd.read_excel(struct_table_file, index_col=0)

        if method=='SPIRAL (Jaccard index=0.05)':
            struct_list = filter_struct_lst_for_result_panel(data_n=data_n, 
                                                             new_Jaccard_thr_genes=0.05)
            struct_list_str = str(struct_list)

        elif method=='SPIRAL (Jaccard index=0.5)':
            struct_list = filter_struct_lst_for_result_panel(data_n=data_n, 
                                                             new_Jaccard_thr_genes=0.5)
            struct_list_str = str(struct_list) 

        elif method=='SPIRAL (Jaccard index=0.75)':
            struct_list = filter_struct_lst_for_result_panel(data_n=data_n, 
                                                             new_Jaccard_thr_genes=0.75)
            struct_list_str = str(struct_list) 

        method_cluster_table = struct_table.loc[struct_list, ['num_genes_in_struct', 'genes']].rename(columns={'num_genes_in_struct': '#genes'})
        method_cluster_table.loc[:, 'genes'] = [str(a.split(',')) for a in method_cluster_table.loc[:, 'genes']]

    if not flag:
        print('\n\n', dataset, '+', method, flag, '!!!!!!!!!!!\n\n')
    if flag:
        print(dataset, '+', method, '\n', method_cluster_table.head())

        #module_GO_terms[dataset][method] = dict()
        GO_terms = dict()

        cluster_table = method_cluster_table
        geneslist = []
        for clus_ind, clus in enumerate(cluster_table.index):
            # read cluster gene list
            if method == "nsNMF" and (data_n in range(20, 81)):
                clustergeneslist = cluster_table.loc[clus, 'genes'].split(", ")
            else:
                clustergeneslist = cluster_table.loc[clus, 'genes'].replace("['", "").replace("']", "").split("', '")

            geneslist = geneslist + clustergeneslist
        
        n_participating_genes[method] = len(set(geneslist))


sagittal-posterior section of a mouse brain
sagittal-posterior section of a mouse brain spatial SPIRAL (Jaccard index=0.05)
filter_struct_lst_for_result_panel!!!
sagittal-posterior section of a mouse brain + SPIRAL (Jaccard index=0.05) 
     #genes                                              genes
1      321  ['Tmeff2', 'Plcl1', 'Cyp20a1', 'Tmbim1', 'Cyp2...
2      507  ['Atp6v1h', 'Stau2', 'Ogfrl1', 'Actr1b', 'Lonr...
6       61  ['Pantr1', 'Stum', 'Nsmf', 'Lhx2', 'Slc1a2', '...
15     906  ['Pcmtd1', 'Gm28784', 'Fam168b', 'Cnnm4', 'Cnn...
25     130  ['Ogfrl1', 'Fam126b', 'D130058E05Rik', 'Ndufa1...
sagittal-posterior section of a mouse brain spatial SPIRAL (Jaccard index=0.5)
filter_struct_lst_for_result_panel!!!
sagittal-posterior section of a mouse brain + SPIRAL (Jaccard index=0.5) 
    #genes                                              genes
1     321  ['Tmeff2', 'Plcl1', 'Cyp20a1', 'Tmbim1', 'Cyp2...
2     507  ['Atp6v1h', 'Stau2', 'Ogfrl1', 'Actr1b', 'Lonr...
3     129  ['Ac

In [14]:
n_participating_genes

{'SPIRAL (Jaccard index=0.05)': 2632,
 'SPIRAL (Jaccard index=0.5)': 6731,
 'SPIRAL (Jaccard index=0.75)': 7269,
 'Seurat': 17599,
 'Hotspot (n_neighbors=30)': 4899,
 'Hotspot (n_neighbors=300)': 8462,
 'nsNMF': 2220,
 'SpatialDE (C=3)': 12405,
 'SpatialDE (C=5)': 12405,
 'SingleCellHaystack (k=3)': 1000,
 'SingleCellHaystack (k=5)': 1000}